The big picture idea here is that I want to build a machine learning model that can predict when a county will have a high (>75th percentile) rate of COVID fatalities. Then I will pull out the most important factors of that model. We have a lot of data that is moderately correlated with COVID fatality rates and often highly correlated with each other. So the machine learning model will help tease out what is really important within the raw data, in a way that minimizes my own biases.

First, a bunch of imports:

In [15]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import roc_auc_score
from COVID_data import prepare_model_data

from sklearn.linear_model import Lasso, LogisticRegressionCV, LassoLarsCV, ElasticNetCV
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import pprint

class config:
    USE_CACHE = True
    CACHE_DIR = "/Users/caseydurfee/msds/data_mining_final_project/cache"

from COVID_data import all_data
data = all_data.get_all_data(config)

Let's look at correlations in year 1 of the pandemic versus year 2 and see if there were changes.  Note that we are looking at all factors, including ones with large rates of missing data, which will artificially inflate those correlation scores. Some of these factors won't be used by the model, because I am throwing out any fields where more than 5% of counties have missing data.

I will get the top 20 factors for each year, filtering out correlations with other death rates.

In [38]:
death_rate_corr = data.corr()['DEATH_RATE_FIRST_YEAR']

# omicron, alpha, delta, etc. death rates are not interesting here
death_cols = list(data.filter(regex = 'DEATH'))

y1_corr = death_rate_corr.reindex(death_rate_corr.abs().sort_values().index) \
    .drop(death_cols)

print(y1_corr[-10:].to_string())

% Some College                      -0.256085
% Free or Reduced Lunch              0.261195
% Uninsured (SVI)                    0.262803
Per Capita Income (SVI)             -0.264968
% Fair/Poor                          0.272457
Life Expectancy                     -0.284754
Infant Mortality Rate                0.285151
MEDIAN_FAMILY                       -0.285368
% Uninsured                          0.291818
% Children in Poverty                0.292585
% Physically Inactive                0.298508
Age-Adjusted Mortality               0.304026
Child Mortality Rate                 0.305280
Years of Potential Life Lost Rate    0.311401
Homicide Rate                        0.319756
% No HS Diploma (SVI)                0.321150
MV Mortality Rate                    0.336547
% Disconnected Youth                 0.337717
Teen Birth Rate                      0.350611
Age-Adjusted Mortality (Hispanic)    0.376489


In [39]:
death_rate_corr = data.corr()['DEATH_RATE_SECOND_YEAR']

death_cols = list(data.filter(regex = 'DEATH'))

y2_corr = death_rate_corr.reindex(death_rate_corr.abs().sort_values().index) \
    .drop(death_cols)

print(y2_corr[-10:].to_string())

% Physically Inactive                0.404044
% Excessive Drinking                -0.414861
Mentally Unhealthy Days              0.418606
REPUB_PARTISAN                       0.419108
Partial Coverage                    -0.420090
Per Capita Income (SVI)             -0.424035
Firearm Fatalities Rate              0.427629
% Frequent Mental Distress           0.431077
% Frequent Physical Distress         0.431779
% Diabetic                           0.432851
Booster Coverage                    -0.433765
Complete Coverage                   -0.439520
% Some College                      -0.442925
Teen Birth Rate                      0.450424
Physically Unhealthy Days            0.452853
MEDIAN_FAMILY                       -0.457310
Years of Potential Life Lost Rate    0.466963
% Disabled (SVI)                     0.482125
Life Expectancy                     -0.490274
Age-Adjusted Mortality               0.495016


Some things to note here:
* there are some metrics that are extremely similar (Age-adjusted Mortality is basically the inverse of Life Expectancy). That's to be expected since we're going to let the model decide which one is the best for predicting high COVID fatality rates.

* vaccination rates don't predict year one fatality rate, which makes sense, given time only flows in one direction. They do show up as a major factor in year two, as expected.

* The correlations in year 2 are all much stronger than year 1. This implies year 2 death rates were more predictable based on data we had before the pandemic (and hence the deaths were more preventable).

We'll start by evaluating models on the data over the entire pandemic, then see if we can do better by splitting up year 1 and year 2.

I am going to test a wide range of models and we'll go with whatever performs best.  Using ROC AUC as the scoring metric since we have unbalanced classes.

The data is going to be limited to counties with a population over 50,000, to be able to pull the most number of factors without doing too much imputation.

In [29]:
SEED = 2718
ITERS = 20000

test_model_classes = [ 
    RandomForestClassifier,
    SVC, 
    LogisticRegressionCV, 
    RidgeClassifierCV, 
    AdaBoostClassifier, 
    BaggingClassifier,
    GradientBoostingClassifier,
    LinearSVC,
    LassoLarsCV,
    ElasticNetCV,
]

test_models = []

## a little voodoo to pass arguments to the constructors that take them.
import inspect
for k in test_model_classes:
    f_args = {}
    sig = inspect.signature(k.__init__)
    if 'random_state' in sig.parameters:
        f_args['random_state'] = SEED
    if 'max_iter' in sig.parameters:
        f_args['max_iter'] = ITERS
    test_models.append(k(**f_args))

roc_auc_scorer = make_scorer(roc_auc_score)

In [76]:
df, X, y = prepare_model_data.make_train_test(data, year=False, min_pop=50000, split=False)

roc_scores = {}
max_roc = 0
for model in test_models:
    model_name = repr(model.__class__)

    mean_score = cross_val_score(model, X, y, scoring=roc_auc_scorer).mean()

    if model_name not in roc_scores:
        roc_scores[model_name] = 0.0
    roc_scores[model_name] += mean_score
    if mean_score > max_roc:
        best_model = model
        max_roc = mean_score

sz = pd.Series(roc_scores)
print(sz.sort_values())

/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWa

<class 'sklearn.linear_model._ridge.RidgeClassifierCV'>            0.655402
<class 'sklearn.ensemble._bagging.BaggingClassifier'>              0.661033
<class 'sklearn.linear_model._logistic.LogisticRegressionCV'>      0.661305
<class 'sklearn.svm._classes.LinearSVC'>                           0.662727
<class 'sklearn.svm._classes.SVC'>                                 0.673727
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>     0.690268
<class 'sklearn.ensemble._forest.RandomForestClassifier'>          0.701608
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>          0.723754
<class 'sklearn.linear_model._least_angle.LassoLarsCV'>            0.815017
<class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>    0.816537
dtype: float64


/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


Now, let's try training models versus year 1 and 2 individually.

In [78]:

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning, FutureWarning

## taken from https://stackoverflow.com/questions/53784971/how-to-disable-convergencewarning-using-sklearn

@ignore_warnings(category=FutureWarning)
@ignore_warnings(category=ConvergenceWarning)
def fit_yearly_models():
    ITERS = 1 # to run test multiple times & take averages.
    for year in [1,2]:
        df, X, y = prepare_model_data.make_train_test(data, year=year, min_pop=50000, split=False)

        roc_scores = {}
        max_roc = 0.0
        best_model = None
        for i in range(ITERS):
            for model in test_models:
                model_name = repr(model.__class__)
                
                mean_score = cross_val_score(model, X, y, scoring=roc_auc_scorer).mean()

                if model_name not in roc_scores:
                    roc_scores[model_name] = 0.0
                roc_scores[model_name] += mean_score
                if mean_score > max_roc:
                    best_model = model
                    max_roc = mean_score

        sz = pd.Series(roc_scores)
        print(f">>>> mean scores for year {year}")
        print(sz / ITERS)

fit_yearly_models()


/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWa

>>>> mean scores for year 1
<class 'sklearn.ensemble._forest.RandomForestClassifier'>          0.600116
<class 'sklearn.svm._classes.SVC'>                                 0.588568
<class 'sklearn.linear_model._logistic.LogisticRegressionCV'>      0.584277
<class 'sklearn.linear_model._ridge.RidgeClassifierCV'>            0.613578
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>     0.644440
<class 'sklearn.ensemble._bagging.BaggingClassifier'>              0.572464
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>          0.605216
<class 'sklearn.svm._classes.LinearSVC'>                           0.636336
<class 'sklearn.linear_model._least_angle.LassoLarsCV'>            0.740688
<class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>    0.739316
dtype: float64


/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWa

>>>> mean scores for year 2
<class 'sklearn.ensemble._forest.RandomForestClassifier'>          0.687254
<class 'sklearn.svm._classes.SVC'>                                 0.680794
<class 'sklearn.linear_model._logistic.LogisticRegressionCV'>      0.714251
<class 'sklearn.linear_model._ridge.RidgeClassifierCV'>            0.693006
<class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>     0.696235
<class 'sklearn.ensemble._bagging.BaggingClassifier'>              0.697728
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>          0.712427
<class 'sklearn.svm._classes.LinearSVC'>                           0.715096
<class 'sklearn.linear_model._least_angle.LassoLarsCV'>            0.865199
<class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>    0.865058
dtype: float64


/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


Looks like ElasticNetCV and LassoLarsCV are our big winners. They both apply regularization, which punishes models for being too complex. This is perfect for our purposes, since we're trying to pull out the top factors that matter the most.

Because there's not a huge difference between them, I selected LassoLarsCV for the final models. It produces simpler models than ElasticNetCV and doesn't have trouble converging on a solution.

now, let's use permutation importance to figure out what really matters to these models that we've built.

In [45]:
from sklearn.inspection import permutation_importance

df, X, y = prepare_model_data.make_train_test(data, year=1, split=False)

best_model = LassoLarsCV(normalize=False).fit(X, y)

result = permutation_importance(best_model, X, y, 
            n_repeats=100)

y1_importance = pd.Series(result.importances_mean, index=df.columns)

disp = y1_importance.reindex(y1_importance.abs().sort_values().index)

print("Year one")
print(disp[disp > 0])

Year one
% Insufficient Sleep     0.000886
% Female                 0.000934
% Rural                  0.000949
MV Mortality Rate        0.001100
% Alcohol-Impaired       0.008845
% No Vehicle (SVI)       0.016103
Income Ratio             0.016110
% No HS Diploma (SVI)    0.021963
% Over 65 (SVI)          0.028022
% Hispanic               0.029458
% Physically Inactive    0.121710
dtype: float64


In [60]:
df, X, y = prepare_model_data.make_train_test(data, year=2, split=False)

best_model = LassoLarsCV(normalize=False).fit(X, y)

result = permutation_importance(best_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_importance = perm_importances.reindex(perm_importances.abs().sort_values().index)

print("Year two")
print(y2_importance[y2_importance > 0][-20:])

Year two
% Physically Inactive               0.000887
% Smokers                           0.000939
% African American                  0.001270
Preventable Hosp. Rate              0.001553
Physically Unhealthy Days           0.001971
MV Mortality Rate                   0.002001
% Rural                             0.002234
% Asian                             0.002713
Complete Coverage                   0.003840
% Female                            0.004449
% Vaccinated                        0.004807
Injury Death Rate                   0.008141
Age-Adjusted Mortality              0.010133
Partial Coverage                    0.012491
% Over 65 (SVI)                     0.020028
% Uninsured (SVI)                   0.025417
Teen Birth Rate                     0.033442
% Single Parent Households (SVI)    0.035241
% Disabled (SVI)                    0.038654
REPUB_PARTISAN                      0.070278
dtype: float64


These importances don't tell us the direction of the correlation (whether they increase or decrease the likelihood of a county having high COVID rates.) We can unite this data with the correlation data to show all factors that mattered during the pandemic.

In [61]:
summary = y1_importance.rename('Y1_IMPORTANCE').to_frame()\
    .join(y1_corr.rename("Y1_CORR"))\
    .join(y2_importance.rename('Y2_IMPORTANCE'))\
    .join(y2_corr.rename("Y2_CORR"))

## todo: put ranks in for importance of factors (most important=1, etc.)


summary[(summary.Y1_IMPORTANCE > 0) | (summary.Y2_IMPORTANCE > 0)].sort_values(by='Y2_IMPORTANCE', ascending=False)

,Y1_IMPORTANCE,Y1_CORR,Y2_IMPORTANCE,Y2_CORR
REPUB_PARTISAN,0.000000,0.136991,0.070278,0.419108
% Disabled (SVI),0.000000,0.090178,0.038654,0.482125
% Single Parent Households (SVI),0.000000,0.191830,0.035241,0.025415
Teen Birth Rate,0.000000,0.350611,0.033442,0.450424
% Uninsured (SVI),0.000000,0.262803,0.025417,0.253985
% Over 65 (SVI),0.028022,0.061256,0.020028,0.239974
Partial Coverage,0.000000,-0.146361,0.012491,-0.420090
Age-Adjusted Mortality,0.000000,0.304026,0.010133,0.495016
Injury Death Rate,0.000000,0.126617,0.008141,0.361140
% Vaccinated,0.000000,-0.167147,0.004807,-0.223298


There are very few factors that are important in both the year one and year two models:

In [75]:
summary[(summary.Y1_IMPORTANCE > 0) & (summary.Y2_IMPORTANCE > 0)].sort_values(by='Y2_IMPORTANCE', ascending=False)

,Y1_IMPORTANCE,Y1_CORR,Y2_IMPORTANCE,Y2_CORR
% Over 65 (SVI),0.028022,0.061256,0.020028,0.239974
% Female,0.000934,0.008388,0.004449,0.031939
% Rural,0.000949,0.102122,0.002234,0.254303
MV Mortality Rate,0.001100,0.336547,0.002001,0.384732
% Physically Inactive,0.121710,0.298508,0.000887,0.404044
% Alcohol-Impaired,0.008845,-0.041038,0.000713,-0.079054
% Insufficient Sleep,0.000886,0.073121,0.000681,0.261217


Another way to look at what's changed is the difference in correlation between year one and year two. In some cases, the correlation went up, and others it went down.

In [59]:
summary['CORR_DIFF'] = abs(summary['Y2_CORR'] - summary['Y1_CORR'])

summary.sort_values(by='CORR_DIFF', ascending=False)[:10]

,Y1_IMPORTANCE,Y1_CORR,Y2_IMPORTANCE,Y2_CORR,CORR_DIFF
Mentally Unhealthy Days,0.0,0.015366,0.000000,0.418606,0.403240
% Disabled (SVI),0.0,0.090178,0.039334,0.482125,0.391947
Physically Unhealthy Days,0.0,0.114035,0.001840,0.452853,0.338818
Firearm Fatalities Rate,0.0,0.103636,0.000367,0.427629,0.323993
% Frequent Mental Distress,0.0,0.109787,0.000000,0.431077,0.321290
% Minority (SVI),0.0,0.230978,0.000000,-0.065145,0.296123
% Non-Hispanic White,0.0,-0.230316,0.000000,0.063103,0.293418
HIV Prevalence Rate,0.0,0.200919,0.000000,-0.086887,0.287806
REPUB_PARTISAN,0.0,0.136991,0.071253,0.419108,0.282117
Partial Coverage,0.0,-0.146361,0.012298,-0.420090,0.273729


Let's see if ElasticNetCV would give us different factors.

In [64]:
df, X, y = prepare_model_data.make_train_test(data, year=2, split=False)

best_model = ElasticNetCV().fit(X, y)

result = permutation_importance(best_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_enet = perm_importances.reindex(perm_importances.abs().sort_values().index)

In [74]:
enet_vs_lars = y2_enet.rename('Y2_ENET').to_frame()\
    .join(y2_importance.rename('Y2_LARS'))

enet_vs_lars['ENET_RANK'] = enet_vs_lars['Y2_ENET'].rank(ascending=False)
enet_vs_lars['LARS_RANK'] = enet_vs_lars['Y2_LARS'].rank(ascending=False)

enet_vs_lars["RANK_CHANGE"] = abs(enet_vs_lars['ENET_RANK'] - enet_vs_lars['LARS_RANK'])


enet_vs_lars[(enet_vs_lars.Y2_ENET > 0) | (enet_vs_lars.Y2_LARS > 0)].sort_values(by=['RANK_CHANGE', 'ENET_RANK'])

,Y2_ENET,Y2_LARS,ENET_RANK,LARS_RANK,RANK_CHANGE
REPUB_PARTISAN,0.065299,0.070278,1.0,1.0,0.0
% Disabled (SVI),0.036214,0.038654,2.0,2.0,0.0
% Single Parent Households (SVI),0.035516,0.035241,3.0,3.0,0.0
Teen Birth Rate,0.031357,0.033442,4.0,4.0,0.0
% Uninsured (SVI),0.020339,0.025417,5.0,5.0,0.0
% Over 65 (SVI),0.019941,0.020028,6.0,6.0,0.0
Partial Coverage,0.011974,0.012491,7.0,7.0,0.0
Age-Adjusted Mortality,0.007333,0.010133,8.0,8.0,0.0
Injury Death Rate,0.006611,0.008141,9.0,9.0,0.0
% Vaccinated,0.004597,0.004807,10.0,10.0,0.0


It looks like the rankings are remarkably consistent between the two models. LARS pick one fewer factor, but the order is almost identical.

In [73]:
sum(enet_vs_lars['Y2_ENET'])

0.27040923332496974

Let's try a couple of the less accurate models and see what factors they deem important. 

First off, let's try LogisticRegressionCV. Because it doesn't have a regularization penalty, it will use all 80 factors in the model, so we will restrict to only the top 20.

In [95]:
lrcv_model = LogisticRegressionCV(max_iter=20000).fit(X, y)

result = permutation_importance(lrcv_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_lrcv = perm_importances.reindex(perm_importances.abs().sort_values().index)

In [96]:
print(y2_lrcv[-20:])

Health_Behaviors_Percentile             0.011556
Segregation index                       0.012210
Firearm Fatalities Rate                 0.013112
Social_&_Economic_Factors_Percentile    0.014796
Length_of_Life_Percentile               0.015107
% Limited English (SVI)                 0.015483
% Single-Parent Households              0.018122
MV Mortality Rate                       0.019517
% No HS Diploma (SVI)                   0.021009
% Asian                                 0.021652
Age-Adjusted Mortality                  0.023863
Physically Unhealthy Days               0.027371
% Insufficient Sleep                    0.032929
% Uninsured (SVI)                       0.043584
Life Expectancy                         0.044399
% Minority (SVI)                        0.045461
% Over 65 (SVI)                         0.057436
Per Capita Income (SVI)                 0.059077
Years of Potential Life Lost Rate       0.079399
REPUB_PARTISAN                          0.087049
dtype: float64


Let's look at the top 20 factors for LARS versus LRCV. How much overlap is there?

In [91]:
enet_vs_lars[-20:].index.intersection(y2_lrcv[-20:].index)


Index(['Years of Potential Life Lost Rate', 'MV Mortality Rate', '% Female',
       '% Over 65 (SVI)', '% Uninsured (SVI)',
       '% Single Parent Households (SVI)', 'REPUB_PARTISAN'],
      dtype='object')

In [92]:
enet_vs_lars[-10:].index.intersection(y2_lrcv[-10:].index)

Index(['% Over 65 (SVI)', '% Uninsured (SVI)', 'REPUB_PARTISAN'], dtype='object')

In [ ]:
svc_model = LinearSVC(max_iter=20000).fit(X,y)

result = permutation_importance(svc_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_svc = perm_importances.reindex(perm_importances.abs().sort_values().index)

In [100]:
enet_vs_lars[-20:].index.intersection(y2_svc[-20:].index)

Index(['Years of Potential Life Lost Rate', 'Physically Unhealthy Days',
       'MV Mortality Rate', '% Asian', 'Age-Adjusted Mortality',
       '% Over 65 (SVI)', '% Uninsured (SVI)', 'REPUB_PARTISAN'],
      dtype='object')

In [101]:
enet_vs_lars[-10:].index.intersection(y2_svc[-10:].index)

Index(['% Over 65 (SVI)', '% Uninsured (SVI)', 'REPUB_PARTISAN'], dtype='object')

So, all four models we trained on pandemic had 3 factors in common among their top factors: '% Over 65 (SVI)', '% Uninsured (SVI)', 'REPUB_PARTISAN'. 

All 4 models identified 'REPUB_PARTISAN' as the strongest factor for predicting whether a county would have a high COVID fatality rate.

So far we have only been looking at counties with a population of over 50,000. Let's train a model on all counties, using only the factors that have wide coverage (less than 5% imputation required.)

In [ ]:
df, X, y = prepare_model_data.make_train_test(data, year=2, split=False, min_pop=None)

In [106]:
df.shape

(2919, 70)

In [107]:
data.shape

(2919, 100)

In [110]:
(cols_to_drop, cols_to_impute, missing_counts) = prepare_model_data.get_cols_to_drop_and_impute(data)

In [112]:
data.shape

(2919, 100)